In [6]:
'''
06/06/25
for now this is a notebook, but if the plan is to automate,
it will become a .py file that can be executed by terminal
12/06/25
All the working functions are now in video_processing_tools.py
'''

# NB - HAS TROUBLE WRITING TO ONEDRIVE
import cv2
import os
import re
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML, display, clear_output

import video_processing_tools as vp


def get_relative_COR(p1a, p2a, centre, frame):
    '''input: p1a, p2a, centre, frame, output: draws lines and returns
    [length_along, length_perp].'''
    p1a = np.array(p1a)
    p2a = np.array(p2a)
    probe_line = p2a - p1a     # direction vector of line A
    if probe_line[0] == 0.0 and probe_line[1] == 0.0:
        return [0, 0]  # handles error where probe_line = 0. Is this from rematch() ?

    length_line = np.sqrt( (p2a[0] - p1a[0])**2 + (p2a[1] - p1a[1])**2 )

    centre = np.array(centre)  # centre point


    length_p1 = np.sqrt( (centre[0] - p1a[0])**2 + (centre[1] - p1a[1])**2 )
    length_p2 = np.sqrt( (centre[0] - p2a[0])**2 + (centre[1] - p2a[1])**2 )

    if length_p1 < length_p2:
        point_p = np.array(p1a)
    else:
        point_p = np.array(p2a)
        probe_line = -probe_line

    cv2.circle(frame, point_p, 5, (205, 0, 205), -1)

    # vector from centre to point on the line
    vec = centre - point_p
    
    # projection of vector onto probe line
    length_along = (np.dot(vec, probe_line) / np.dot(probe_line, probe_line))

    # location of the foot along the line relative to point 2a
    vec_parallel = length_along * probe_line
    
    # foot of the perpendicular (intersection point on line A)
    point_c = point_p + vec_parallel
    point_c = [int(point_c[0]), int(point_c[1])]

    # distance from COR to the foot
    length_perp = np.sqrt( (point_c[0] - centre[0])**2 + (point_c[1] - centre[1])**2 )

    # writing to the frame
    # cv2.circle(img, centre, radius, colour, thickness=-1 fills it)
    # cv2.line(frame, p1, p2, (RGB), thickness)
    cv2.circle(frame, point_c, 4, (165, 0, 0), -1)
    cv2.line(frame, point_c, centre, (165, 165, 0), 3)
    
    return [length_along, length_perp/length_line]
            

In [7]:
def main_COR(inpath, outpath, skip_frame=1, start_time=0, end_time=None,
             validate=False, rel_COR=True):
    '''input: file to analyse, output: path to analysed video. optional:
    number of frames to skip to get centre, seconds of video to skip.'''
    # loading input
    if os.path.exists(inpath):
        cap = cv2.VideoCapture(inpath)
    else:
        print("main_COR: input file does not exist")
        return
    
    # matches the new video to the old video specifications
    # 21-11-25
    # the fps of the slow motion videos does not correspond to real time,
    # so the values of angular velocity are WRONG by the factor it has been slowed.
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_num = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frame_num / fps

    # THE PROBLEM WITH ROTATION SPEED IS THE SLOW MOTION FRAME RATE =! PLAYBACK FPS.
    print('duration', duration, 'fps', fps, 'frame num', frame_num)
    input_info = (fps, width, height)

    # skips ahead ||| MAKE SURE PROBE IS VISIBLE FROM THE FILM START ||| big obstacle for automation
    start_frame = int(start_time * fps)

    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # saving output to mp4 format
    video_out = vp.video_writer(outpath, input_info)

    # needed for rotational speed calc
    frame_info = []
    rot_list = []
    rot_speeds = []
    timestamps = []
    relative_centres = []
    frame_count = 0

    if end_time is not None:
        end_frame = int(end_time * fps)
    else:
        end_frame = 100000  # any large number
    
    while cap.isOpened() and frame_count <= end_frame:
        ret, frame = cap.read()
        # end of video
        if not ret:
            break
        
        # filtering for red and then using the binary mask ! NOT NEEDED FOR FIT_RECTANGLE
        # mask = colour_mask(frame, colour='red')
# =====================================================================
# ================== Choose depending on the problem ==================
        # p1, p2 = detect_line(mask)
        # p1, p2 = fit_line(mask)
        p1, p2 = vp.fit_rectangle(frame)
        
        if p1 and p2 is not None:
            p1 = np.array(p1)
            p2 = np.array(p2)

            # just for tracking really not needed
            mid = vp.midpoint(p1, p2)

            # storing this info for use in next part
            frame_info.append((frame_count, p1, p2))
            
# ============ COMPARING WITH PREVIOUS FRAMES TO FIND COR ==========
# A and B refer to time, 1 and 2 to endpoints again
            if len(frame_info) >= skip_frame + 1:
                # takes the info stored in the frame one skip ago
                frameA_count, p1a, p2a = \
                    frame_info[-(skip_frame + 1)]
                
                # compares to last frame (B is further ahead in time)
                frameB_count, p1b, p2b = \
                    frame_info[-1]
                
                # we will see if this matches the correct points based on length
                p1a, p2a, p1b, p2b = vp.rematch(p1a, p2a, p1b, p2b)  

                # p1a, p2a, p1b, p2b
                centre = vp.get_COR(p1a, p2a, p1b, p2b, frame)
                speed = vp.rotation_speed(p1a, p2a, p1b, p2b, frameB_count, \
                                       frameA_count, fps)

# checks centre was calculated, is a coord, and is in the picture frame.
                if centre is not None and len(centre) == 2 and vp.in_frame(centre, width, height) is True:
                    rot_list.append(centre)
                    rot_speeds.append(speed)
                    timestamps.append((frame_count - start_frame) / fps)

# cv2.circle(img, centre, radius, colour, thickness=-1 fills it)
# cv2.line(frame, p1, p2, (RGB), thickness)
                    # drawing previous line (darker red)
                    cv2.line(frame, p1a, p2a, (0, 0, 165), 3)
                    # midpoint
                    cv2.circle(frame, tuple(mid.astype(int)), 4, 
                                (0, 225, 225), -1)
                    # COR
                    cv2.circle(frame, centre, 6,
                                (0, 255, 0), 4)
                    # mapping lines
                    cv2.line(frame, p1a, p1b, (0, 255, 0), 3)
                    cv2.line(frame, p2a, p2b, (0, 255, 0), 3)
                    
                    #
                    if rel_COR is True:
                        rel_centre = get_relative_COR(p1b, p2b, centre, frame)
                        relative_centres.append(rel_centre)

                else:
                    # print("get_COR returned None")
                    exit()
                    
        video_out.write(frame)
            
        frame_count = frame_count + 1

    video_out.release()
    cap.release()
    print(f"Video saved as {outpath}")

    # Plot
    plt.plot(timestamps, rot_speeds)
    plt.xlabel('Time (s)')
    plt.ylabel('Angular velocity (rad/s)')
    plt.title('Rotational Velocity Over Time')
    plt.show()

    if rel_COR is True:
        x_mean = np.mean([x[0] for x in relative_centres])
        y_mean = np.mean([x[1] for x in relative_centres])

        plt.scatter(x_mean, y_mean, color="red", marker='o', label=f"Mean Centre of Rotation \nx = {x_mean:.3f}, y = {y_mean:.3f}")# , facecolors='none')
        plt.scatter([x[0] for x in relative_centres], [x[1] for x in relative_centres], s=2)
       
        plt.xlim(-0.6, 0.6)
        plt.ylim(0.6, -0.6)
        plt.xlabel('length along tape')
        plt.ylabel('length of perpendicular line to COR')
        plt.title('Position of the COR relative to the tape')
        plt.legend()
        plt.show()
    
    if validate is True: 
        centres = np.array(rot_list)
        return centres




In [ ]:
import datetime  # to avoid overwriting files
time = datetime.datetime.now()
timestamp = time.strftime("%m%d_%H%M%S")

# ifile = r"C:\Users\rj429\test_line_video.mp4"           # default simulated line video name
# ifile = r"C:\Users\rj429\M3R1S_145657152.mp4"           # thick red tape. start~3s
# ifile = r"C:\Users\rj429\M1B_R1S_145621480.mp4"         # thin tape diagonal
# ifile = r"M3R2S_155427.mp4"                             # thick red tape
# ifile = r"C:\Users\rj429\M1B_EXTENDED_UR1_154519.mp4"   # blue probe
# ifile = "M1B_EXTENDED_UR1_154519.mp4"
# ifile = r"C:\Users\rj429\M3HR2S_151220.mov"             # new data with blue tape, start~2
# ifile = "M3UHR2SE_151556.mov"              # new data with blue tape, start~2
# ifile = "M2HR2SE_150656.mov"                  # blue tape, diagonally across wingspan
ifile = "M3UHR2SE_151556.mov"

ifile = "test"


ofile = r"C:\Users\rj429\test_data.mp4"
#try:
strparts = ifile.split("_")
case = strparts[0]
ofile = f"results/result__{case}_{timestamp}.mp4"
# except:
    # print("Error: no good input")

# main_COR(ifile, ofile, skip_frame=1, start_time=12, end_time=18)
# vp.mask_check(ifile, ofile, start_time=8)
# vp.test_data(ofile, centre=[205, 235], thickness=2, amplitude=0)
main_COR(ifile, ofile, start_time=0.5, end_time=3, skip_frame=3)
# validate(ifile, ofile, amplitude=7)



main_COR: input file does not exist


AttributeError: module 'video_processing_tools' has no attribute 'validate'